In [46]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.font_manager import FontProperties
from scipy.interpolate import interp1d
import os 
import sys
from CFST_Cir_Section_Analysis import CFST_Cir_Section_Analysis
from DataMonoClass import DataMono



def cutData(data, ratio=0.4, num=20):
    DD, N, M, Nc, Mc, Ns, Ms = data
    threhold = ratio * M.max()
    for i in range(M.size):
        if M[i] >= threhold:
            break
    data = data[:,i:]
    DD, N, M, Nc, Mc, Ns, Ms = data
    f = interp1d(data[0], data)
    DD_new = np.linspace(DD[0], DD[-1], num=num)
    return f(DD_new)

cutRatio = 0.2
iniRatio = 0.6
targetDuctility = 3.5
loadLevel = [0, 0.05, 0.1, 0.2, 0.3]

pathTest = 'D:/Google Drive/Python_Scripts/Test'
path = './Test'

#Load Specimen group
f = open('{0}/Specimen_Group.txt'.format(pathTest), 'r')
groups = [int(item)-1 for item in f.read().split('\n')[0].split(' ')]

f.close()

#Load the specimen data
Conditions = np.loadtxt('{0}/Specimen_Conditions.txt'.format(pathTest))[groups]
#load specimen names
names = np.loadtxt('{0}/file_names.txt'.format(path), dtype='str')

output = np.zeros((1,6))
for i in range(len(names)):
    curData = np.loadtxt('{0}/{1}.out'.format(path, names[i])).T
    curData = cutData(curData, cutRatio, num=100)
    DD, N, M, Nc, Mc, Ns, Ms = curData
    D, temp, t, L, fc, fy, fu, Naxial = Conditions[i]
    curCFST = CFST_Cir_Section_Analysis(D, t, fc, fy, Num=20)
    fccs, hcs = curCFST.getFcAvg(Nc*1000.0, Mc*1.0e6)
    fybs, hss = curCFST.getFyAvg(Ns*1000.0, Ms*1.0e6)
    
    fcData = DataMono(DD, fccs, iniRatio)
    fyData = DataMono(DD, fybs, iniRatio)
    
    temp, fcc = fcData.findYield1()
    temp, fyb = fyData.findYieldByDuctility(targetDuctility)
    
    print(fcc, fyb)
    #fcc = min(fcc, fccs[-1])
    #fig, ax = plt.subplots(1,1)
    #ax.plot(DD, fybs)
    output = np.vstack((output, [D, t, L, fcc, fyb, Naxial]))
output = output[1:]
np.savetxt('./Test/condition_Method.out', output)

27.165540657 327.609019266
23.928830169 319.107909036
29.9137201327 411.309983611
32.7303820303 400.782150472
56.8254278758 414.484518272
49.526198475 406.383505622
80.3435706994 414.790044665
70.185818427 410.018635199
